In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
pd.set_option('max_columns', 81)
pd.set_option('max_rows', 81)

In [3]:
df = pd.read_csv('../train.csv')
profile = ProfileReport(df, title='Profile_Report_Housing01',
                        explorative=True)

In [4]:
def show_stats(series):
    """Print stats from a Pandas Series"""
    
    # Describe
    descriptive_stats = series.describe()
    print('Describe Method:')
    print(descriptive_stats)
    print('---------------------------------------------------')
    
    
    # Numeric Columns only
    if series.dtype == 'int64':
        mean = series.mean()
        maximum = max(series)
        minimum = min(series)
        print(f'''
The Min/Max of column {series.name}: ({minimum}, {maximum})
        
The maximum is {maximum - mean} away from the mean
The minimum is {mean - minimum} away from the mean
        ''')
        print('---------------------------------------------------')
    elif series.dtype == 'O':
        mode = series.mode()
        print(f'The most freqent class is: {mode[0]}')
        print('---------------------------------------------------')
        
        print(f'Value Counts for column: {series.name}')
        print(series.value_counts())
        print('---------------------------------------------------')
    
    
    # Number of NaN values
    nans = series.isna().sum()
    nans_to_percent = nans/len(series * 100)
    print(f'Number of NaNs: {nans}')
    print(f'Percent of Null Values for the column: {nans_to_percent}%')
    print('---------------------------------------------------')
    
    
    # Number of Unique Values - Only display if there are less than 20 unique
    unique = series.unique()
    print(f'There are {len(unique)} values')
    
    if len(unique) <= 20:
        print('Unique Values:')
        print(unique)
        print('---------------------------------------------------')
    else:
        print('Warning: High Cardinality')
        print('---------------------------------------------------')

In [5]:
for col in df.columns:
    if df[col].isna().sum() > 0:
        print(f'Column: {col}')
        print(f'Null Values: {df[col].isna().sum()}')
        print('====================================')

Column: LotFrontage
Null Values: 259
Column: Alley
Null Values: 1369
Column: MasVnrType
Null Values: 8
Column: MasVnrArea
Null Values: 8
Column: BsmtQual
Null Values: 37
Column: BsmtCond
Null Values: 37
Column: BsmtExposure
Null Values: 38
Column: BsmtFinType1
Null Values: 37
Column: BsmtFinType2
Null Values: 38
Column: Electrical
Null Values: 1
Column: FireplaceQu
Null Values: 690
Column: GarageType
Null Values: 81
Column: GarageYrBlt
Null Values: 81
Column: GarageFinish
Null Values: 81
Column: GarageQual
Null Values: 81
Column: GarageCond
Null Values: 81
Column: PoolQC
Null Values: 1453
Column: Fence
Null Values: 1179
Column: MiscFeature
Null Values: 1406


In [6]:
df1 = df.copy()

condition0 = (len(df1) * 0.1)
cols = df1.columns

for col in cols:
    n_NaN = (df1[col].isna().sum())
    if n_NaN > condition0:
        df1.drop(columns=col, inplace=True)
    elif 0 < n_NaN < condition0:
        if df1[col].dtype != 'O':
            if len(df1[col].unique()) > 20:
                df1[col].fillna(value=df1[col].mean(), inplace=True)
            else:
                df1[col].fillna(value=df1[col].mode().values()[0], inplace=True)
        else:
            df1[col].fillna(value=df1[col].mode().values[0], inplace=True)

In [7]:
df1['SalePrice'].mean()

180921.19589041095

In [8]:
print(df1.isna().sum())
df1.head()

Id               0
MSSubClass       0
MSZoning         0
LotArea          0
Street           0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
MasVnrArea       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinSF1       0
BsmtFinType2     0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
KitchenQual      0
TotRmsAbvGrd

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000


## Let's tackle the columns one-by-one

### What can we infer from each column?

- Further breakdown at [http://jse.amstat.org/v19n3/decock/DataDocumentation.txt]

1. Id
    - Column of ids for each listing - only necessary for prediction submission

2. MSSubClass
    - Categorical column pretending to be numeric... pathetic!
        - 15 distinct values
    - Distribution: Mode is 20
        - Values:
            - '20': [536] == 1-Story 1946 & Newer All Styles
            - '60': [299] == 2-Story 1946 & Newer
            - '50': [144] == 1-1/2 Story - Unfinished All Ages
            - '120': [87] == 1-Story (PUD) Planned Unit Development - 1946 & Newer
            - '30': [69] == 1-Story 1945 & Older
            - '160': [63] == 2-Story PUD - 1946 & Newer
            - '70': [60] == 2-Story 1945 & Older
            - '80': [58] == Split or Multi-Lvl
            - '90': [52] == Duplex - All Styles and Ages
            - '190': [30] == 2-Family Conversion - All Styles and Ages
            - '85': [20] == Split Foyer
            - '75': [16] == 2-1/2 Story All Ages
            - '45': [12] == 1-1/2 Story - Unfinished All Ages
            - '180': [10] == PUD - Multilevel - Inclined Split Lev/Foyer
            - '40': [4] == 1-Story w/Finished Attic All Ages
    - On further review, this could present some data leakage if you combine with HouseStyle - maybe extract the age of the dwelling from this column

3. MSZoning
    - Categorical - Zoning classification
        - 5 distinct values
    - Distribution: Mode is 'RL' and is 79% of observations
        - Values:
            - 'RL': [1151] == Residential Low Density
            - 'RM': [218] == Residential Medium Density
            - 'FV': [65] == Floating Village Residential
            - 'RH': [16] == Residential High Density
            - 'C (all)': [10] == all Commercial

4. LotArea
    - Numeric(continuous) - Square-footage of a lot
    - VERY Widespread - 95th percentile is [17401.15] but Max is [215245]
        - Consider filtering out past 95th

5. Street
    - Categorical - Is is [Paved] or [Gravel]
    - DEFINITELY DROP - Paved is 99.6% of the observations

6. LotShape
    - Categorical - Shape of the lot
        - 4 distinct values
    - Distribution: Mode is [Reg]
        - 'Reg': [925]
        - 'IR1': [484]
        - 'IR2': [41]
        - 'IR3': [10]
    - Since most of the values seem to be in [Reg] and [IR1], consider consolidating all IRs into one value [IR]

7. LandContour
    - Categorical - Eveness of the property
        - 4 Distinct values
    - Distribution: Mode is [Lvl] and represents ~90% of the values
        - 'Lvl': [1311] == Level
        - 'Bnk': [63] == Banked
        - 'HLS': [50] == Hillside
        - 'Low': [36] == Depression
    - Consider classification as either [Lvl] or [not_Lvl]

8. Utilities
    - Categorical - Are utilities included?
    - DEFINITELY DROP - [AllPub] is 99.9% of the observations

9. LotConfig
    - Categorical - Most likely Parking Lot type
        - 5 distinct values
    - Distribution - Mode is [Inside] and 72.1% of the observations
        - 'Inside': [1052]
        - 'Corner': [263]
        - 'CulDSac': [94]
        - 'FR2': [47]
        - 'FR3': [4]
    - Consider combining FR2 and FR3 - They are essentially the same

10. LandSlope
    - Categorical - The slope of the property
        - 3 distinct values
    - Distribution - Mode is [Gtl] and is 94.7% of the observations
        - 'Gtl': [1382] == Gentle Slope
        - 'Mod': [65] == Moderate Slope
        - 'Sev': [13] == Severe Slope
    - Most likely DROP however, might be able to keep it around - do some tests!

11. Neighborhood
    - Categorical - Various neighborhood names
        - 25 distinct values
    - Seems pretty important to keep!

12. Condition1/Condition2
    - Categorical - Proximity to main railroad - [2] denotes a second railroad
    - Consider dropping as [Norm] is 86.3% of the observations (more for [2])
    - If you keep one, keep Condition1 as there's a better distribution
        - 'Norm': [1260]
        - 'Feedr': [81] == Adjacent to a feeder street
        - 'Artery': [48] == Adjacent to an arterial street
        - 'RRAn': [26] == Adjacent to North-South railroad
        - 'PosN': [19] == Near positive off-site feature--park, greenbelt, etc.
        - 'RRAe': [11] == Adjacent to East-West railroad
        - 'PosA': [8] == Adjacent to positive off-site feature
        - 'RRNn': [5] == Within 200 feet of N-S railroad
        - 'RRNe': [2] == Within 200 feet of E-W railroad
    - Consider consolidating some values

13. BldgType
    - Categorical - Type of dwelling
        - 5 distince values
    - Distribution - Mode is [1Fam] which is 83.6% of the values
        - '1Fam': [1220] == Single Family
        - 'TwnhsE': [114] == Townhouse End unit
        - 'Duplex': [52] == Duplex
        - 'Twnhs': [43] == Townhouse Inside unit
        - '2fmCon': [31] == Two-Family Conversion - Originally a single-family dwelling
    - May consider consolidating

14. HouseStyle
    - Categorical - Style of dwelling
        - 9 distinct values
    - Distribution - Mode is [1Story] which is 49.7% of all observations
        - '1Story': [726]
        - '2Story': [445]
        - '1.5Fin': [154] == 1-1/2 Story: 2nd Level finished
        - 'SLvl': [65] == Split Level
        - 'SFoyer': [37] == Split Foyer
        - '1.5Unf': [14] == 1-1/2 Story: 2nd Level unfinished
        - '2.5Unf': [11] == 2-1/2 Story: 2nd Level unfinished
        - '2.5Fin': [8] == 2-1/2 Story: 2nd Level finished
    - Consider consolidation:
        - Separate by finished/unfinished

15. OverallQual
    - Categorical(numeric) - Rates overall material and finish of the house
        - 10 distinct values (range)
    - Distribution - Mode is [5] which is 27.2% of observations
        - '5': [397] == Average
        - '6': [374] == Above Average
        - '7': [319] == Good
        - '8': [168] == Very Good
        - '4': [116] == Below Average
        - '9': [43] == Excellent
        - '3': [20] == Fair
        - '10': [18] == Very Excellent
        - '2': [3] == Poor
        - '1': [2] == Very Poor
    - Consider keeping as is OR separate in some fashion such as:
        - Poor - Fair - Average - Good - Excellent
        - Below_4 - Average (4,5,6) - Above_6

16. OverallCond
    - Categorical(numeric) - Rates overall condition of the house
        - 9 unique values
    - Distribution - Mode is [5] which is 56.2% of observations
        - '5': [821] == Average
        - '6': [252] == Above Average
        - '7': [205] == Good
        - '8': [72] == Very Good
        - '4': [57] == Below Average
        - '3': [25] == Fair
        - '9': [22] == Excellent
        - '2': [5] == Poor
        - '1': [1] == Very Poor
    - Consider keeping as is OR separate in some fashion such as:
        - Poor - Fair - Average - Good - Excellent
        - Below_4 - Average (4,5,6) - Above_6

17. YearBuilt
    - Categorical(numeric) - Original Construction date
    - Distribution - Left-Skew
        - Range from 1872 to 2010
    - Consider Divvying by decades or specific date (Some columns divide before and after 1946)

18. YearRemodAdd
    - Categorical(numeric) - Remodel date (Same as construction date if no remodel or addition)
    - Distribution - Local max at 1950 (12.2%) and skewed left
        - Range from 1950 to 2010
    - Consider making a new column called WAS_REMODELED.  If YearBuilt and YearRemodAdd differ, the home was remodeled.

19. RoofStyle
    - Categorical - Type of roof
        - 6 unique values
    - Distribution - Mode is [Gable] with 78.2% of the observations
        - 'Gable': [1141]
        - 'Hip': [286]
        - 'Flat': [13]
        - 'Gambrel': [11]
        - 'Mansard': [7]
        - 'Shed': [2]
    - This is an 'iffy' column.  The trend goes toward two values [Gable] and [Hip] which represent 97.8% of the observations
    
20. RoofMatl
    - Categorical - Material used to build the roof
    - Distribution - Mode is [CompShg] which represents 98.2% of the observations
    - Consider dropping this one.  Could be indicative of the former column

21. Exterior1st/2nd
    - Categorical - Exterior covering on house (2nd == if more than one material)
        - 15 unique values
    - Distribution - Mode is [VinylSd] which represents 35.3% of observations
        - 'VinylSd': [515] == Vinyl Siding
        - 'HdBoard': [222] == Hard Board
        - 'MetalSd': [220] == Metal Siding
        - 'Wd Sdng': [206] == Wood Siding
        - 'Plywood': [108] == Plywood
        - 'CemntBd': [61] == Cement Board
        - 'BrkFace': [50] == Brick Face
        - 'WdShing': [26] == Wood Shingles
        - 'Stucco': [25] == Stucco (plaster)
        - 'AsbShng': [20] == Asbestos Shingles
        - 'BrkComm': [2] == Brick Common
        - 'Stone': [2] == Stone
        - 'CBlock': [1] == Cinder Block
        - 'AsphShn': [1] == Asphalt Shingles
        - 'ImStucc': [1] == Imitation Stucco
    - This is a tricky column - 5 of the classes each represent 0.1% or less of the data and combining them might not make sense - may have to sick with the type of material (wood, vinyl, etc.)

22. MasVnrType
    - Categorical - Masonry Veneer type
        - 4 unique values
    - Distribution - Mode is [None] representing 59.7% of the observations
        - 'None': [872]
        - 'BrkFace': [445]
        - 'Stone': [128]
        - 'BrkCmn': [15]
    - This could be a good column to use however, the previous column is somewhat similar.  This one is easier to break down as 'BrkFace' and 'BrkCmn' can be combined into 'Brick'.

23. MasVnrArea
    - Numeric - Masonry Veneer area - square footage
    - Distribution - Not good, ZEROS represent 59% of observations
    - DEFINITELY DROP this column as we might get more out of the former column

24. ExterQual
    - Categorical - Exterior Quality
        - 4 unique values
    - Distribution - Mode is [TA] which represents 62.1% of the observations
        - 'TA': [906] == Average/Typical
        - 'Gd': [488] == Good
        - 'Ex': [52] == Excellent
        - 'Fa': [14] == Fair
    - Consider consolidation but, might be okay as is

25. ExterCond
    - Categorical - Condition of exterior
    - Consider dropping since most of the values are [TA]

26. Foundation
    - Categorical - Type of material used for foundation
        - 6 unique values
    - Distribution - Mode is [PConc] which represents 44.3% of the observations
        - 'PConc': [647] == Poured Concrete
        - 'CBlock': [634] == Cinder Block
        - 'BrkTil': [146] == Brick & Tile
        - 'Slab': [24]
        - 'Stone': [6]
        - 'Wood': [3]
    - The last 3 values don't represent much so, I am not counting on this column to help much

27. BsmtQual
    - Categorical - Height of basement
        - 4 unique values
    - Distribution - Mode is [TA] which represents 47% of the data
        - 'TA': [686] == Average/Typical (80-89")
        - 'Gd': [618] == Good (90-99")
        - 'Ex': [121] == Excellent (100"+)
        - 'Fa': [35] == Fair (70-79")
    - Consider consolidation but, might be okay as is

28. BsmtCond
    - Categorical - Condition of Basement
    - Similarly to ExterCond, most of the values are [TA] and might be worth dropping

29. BsmtExposure
    - Categorical - Referring to walkout or garden level wall exposure
        - 4 unique values
    - Distribution - Mode is [No] representing 67.9% of the data
        - 'No': [991] == No exposure
        - 'Av': [221] == Average exposure
        - 'Gd': [134] == Good exposure
        - 'Mn': [114] == Minimum exposure
    - There are a fair amount of each class so, I think we could keep this one

30. BsmtFinType1/Type2
    - Categorical - Rating of basement finished area
        - 6 unique values
    - Distribution - Mode is [Unf] representing 32% of the data
        - 'Unf': [467] == Unfinished
        - 'GLQ': [418] == Good living quarters
        - 'ALQ': [220] == Average living quarters
        - 'BLQ': [148] == Below Average living quarters
        - 'Rec': [133] == Average rec room
        - 'LwQ': [74] == Low quality
    - Worth keeping Type1, Type2 has considerably less diversity

31. BsmtFinSF1/SF2
    - Numeric - Basement Finished square footage
    - Distribution
        - SF1 - 32% Zeros - Consider dropping
        - SF2 - 88.6% Zeros - Definitely drop

32. BsmtUnfSF - Only applies to unfinished basements
    - Drop column

33. TotalBsmtSF
    - Numeric - Total basement square footage
    - Distribution
        - Range of 0 - 6110
        - 95th Percentile at 1753
    - Consider filtering out values greater than [1753]

34. Heating
    - Categorical - Type of heading
    - DROP as most values are in one class [GasA]

35. HeatingQC
    - Categorical - Heating quality and condition
        - 5 unique values
    - Distribution - Mode is [Ex] which represents 50.8% of the data
        - 'Ex': [741]
        - 'TA': [428]
        - 'Gd': [241]
        - 'Fa': [49]
        - 'Po': [1]
    - Consider merging [Fa] and [Po] as Heat can be a dealbreaker if suboptimal

36. CentralAir
    - Boolean
    - Distribution - Mode is [Y] representing 93.5% of the data
        - 'Y': [1365]
        - 'N': [95]
    - May have to drop it but, could be worth keeping

37. Electrical
    - Categorical - electrical system
        - 5 unique values
    - Distribution - Mode is [SBrkr] representing 91.4% of the data
        - 'SBrkr': [1335]
        - 'FuseA': [94]
        - 'FuseF': [27]
        - 'FuseP': [3]
        - 'Mix': [1]
    - Consider doing an either/or with [SBrkr/Fuse]

38. 1stFlrSF/2ndFlrSF
    - Numeric - First floor square feet
    - Distribution
        - Range: 334 - 4692
        - Skewed right
        - 95th % at 1831.25
    - DROP 2ndFlrSF as most are 0s - there is already a column indicating a second floor which could lead to data leakage

39. LowQualFinSF
    - Numeric - Low quality finished square footage (all floors)
    - Distribution - 98.2% are Zeros so DROP IT

40. GrLivArea
    - Numeric - Above-Ground living area square footage
    - Distribution
        - Range: 334 - 5642
        - Skewed right
        - 95th % at 2466.1

41. BsmtFullBath/HalfBath
    - Categorical - number of bathrooms basement level
    - Turn into 'HAS_BATH_BSMT'

42. FullBath
    - Categorical - Number of full bathrooms above basement level
    - Distribution - Mode is [2] with 52.6% of the data
        - '2': [768]
        - '1': [650]
        - '3': [33]
        - '0': [9]
    
43. HalfBath
    - Categorical - Number of half bathrooms above basement level
    - Distribution - Mode is [0] with 62.5% of the data
        - '0': [913]
        - '1': [535]
        - '2': [12]
    - Perhaps do 0 or 1+ ?

44. BedroomAbvGr
    - Categorical(numeric) - Number of bedrooms above-ground
        - 8 unique values
    - Distribution - Mode is [3] which is 55.1% of the data - slightly skewed to the right
        - '3': [804]
        - '2': [358]
        - '4': [213]
        - '1': [50]
        - '5': [21]
        - '6': [7]
        - '0': [6]
        - '8': [1]
    - Because there are so few observations at the extremes, maybe do:
        - 0-1, 2, 3, 4+

45. KitchenAbvGr
    - Categorical(numeric) - Kitchens above ground
    - Mostly [1] so, I'd drop this

46. KitchenQual
    - Categorical - Kitchen quality
        - 4 unique values
    - Distribution - Mode is [TA] representing 50.3% of data
        - 'TA': [735]
        - 'Gd': [586]
        - 'Ex': [100]
        - 'Fa': [39]

47. TotRmsAbvGrd
    - Categorical(numeric) - Total rooms above ground (no bathrooms)
    - Distribution:
        - Range: 2-14
        - 95th % at 10
    - Maybe do: (2-5, 6, 7, 8+)

48. Functional
    - Not spread enough

50. Fireplaces
    - Categorical - num fireplaces
    - Dist: Mode is [0] or 47.3% of data
        - '0': [690]
        - '1': [650]
        - '2': [115]
        - '3': [5]
    - Do (0, 1, 2+)

51. GarageType
    - Categorical - type of garage
        - 6 unique values
    - Dist: Mode is [Attchd] which is 65.1% of data
        - 'Attchd': [951]
        - 'Detchd': [387]
        - 'BuiltIn': [88]
        - 'Basement': [19]
        - 'CarPort': [9]
        - '2Types': [6]
    - Include this

52. GarageYrBuild
    - Could cause data leakage with the house built year

53. GarageFinish
    - Categorical - Interior finish of garage
        - 3 distinct
    - Dist: mode is [Unf] representing 47% of data
        - 'Unf': [686]
        - 'RFn': [422]
        - 'Fin': [352]
    - Include this

54. GarageCars
    - Categorical(numeric) - Garage size in car capacity
        - Range 0-4
    - do (0, 1, 2, 3+)

55. GarageArea
    - Similar to former - EXCLUDE

56. GarageQual
    - Skewed to one class

57. GarageCond
    - Skewed to one class

58. PavedDrive
    - Skewed to one class

59. WoodDeckSF
    - Change to 'HasDeck' and report boolean Yes or No

60. NEW CAT: PorchType
    - Combined: - Assuming every other value is 0
        - OpenPorchSF = [804]
        - EnclosedPorch = [208]
        - 3SsnPorch = [24]
        - ScreenPorch = [116]
        - NoPorch = [308]

61. NEW CAT: HasPool
    - From PoolArea:
        - All values above 0 give a [1]

62. MiscVal
    - Skewed - Don't include

63. MoSold
    - Categorical(ordinal) - Month the property sold
    - INCLUDE ALL VALUES

64. YrSold
    - Categorical(numeric) - Year the protery sold
    - Seems evenly split - 2010 has fewer values

65. SaleType
    - Categorical - DONT

66. SaleCondition
    - Categorical - Condition of sale
    - Include all

67. SalePrice
    - Target Variable
    - Numeric - Continuous
    - Distribution:
        - Range: 34900-755000
        - 95th % at 326100 (Less than half the max)
    - Exclude above the 95th percentile

In [9]:
df1.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape',
       'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'Enclos

In [10]:
include_as_is = [
    'MSZoning', 'LandSlope', 'Neighborhood', 'BldgType', 'ExterQual',
    'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'CentralAir', 'FullBath',
    'KitchenQual', 'GarageType', 'GarageFinish', 'SaleCondition', 'OpenPorchSF'
]

some_work_numeric = ['LotArea', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea']

some_work = [
    'LotShape', 'LandContour', 'LotConfig', 'Condition1',
    'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrType',
    'HeatingQC', 'Electrical', 'BsmtFullBath','BsmtHalfBath',
    'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
    'GarageCars', 'WoodDeckSF', 'PoolArea', 'MoSold', 'YrSold'
]

maybes = [
    'BsmtFinSF1', 'HouseStyle', 'OverallCond', 'Foundation',
    'EnclosedPorch', '3SsnPorch', 'ScreenPorch'
]

In [11]:
cleaned = df1.copy()

In [20]:
# Work to be done for some_work columns
# ==========================================================

# Start with the numeric columns filtering out the outliers (> 95-Percentiles)

# 1. LotArea > 95-percentile (17401.15 ftsq)
condition_LotArea = (cleaned['LotArea'] <= 17401.15)

# 2. TotalBsmtSF > 1753
condition_TotalBsmtSF = (cleaned['TotalBsmtSF'] <= 1753)

# 3. 1stFlrSF > 1831.25
condition_1stFlrSF = (cleaned['1stFlrSF'] <= 1831.25)

# 4. GrLivArea > 2466.1
condition_GrLivArea = (cleaned['GrLivArea'] <= 2466.1)

# 5. SalePrice > 326100
condition_SalePrice = (cleaned['SalePrice'] <= 326100)


# ==========================================================

# Next, columnal consolidation

# 1. LotShape - (Combine Irregular)
cleaned.loc[cleaned['LotShape'].str.startswith('IR'), 'RegularLotShape'] = 0
cleaned.loc[cleaned['LotShape'].str.startswith('Reg'), 'RegularLotShape'] = 1

# 2. LandContour - (Combine non Lvl values)
cleaned.loc[(cleaned['LandContour'] == 'Bnk') | (cleaned['LandContour'] == 'HLS') | (cleaned['LandContour'] == 'Low'), 'LandIsLvl'] = 0
cleaned.loc[cleaned['LandContour'] == 'Lvl', 'LandIsLvl'] = 1

# 3. LotConfig - (FR2, FR3 essentially the same)
# Ordinality - {'Inside': 0, 'Corner': 1, 'CulDSac': 2, 'FR': 3}
cleaned.loc[cleaned['LotConfig'] == 'Inside', 'LotConfigCL'] = 0
cleaned.loc[cleaned['LotConfig'] == 'Corner', 'LotConfigCL'] = 1
cleaned.loc[cleaned['LotConfig'] == 'CulDSac', 'LotConfigCL'] = 2
cleaned.loc[cleaned['LotConfig'].str.startswith('FR'), 'LotConfigCL'] = 3

# 4. Condition1 - (Combine adjacency types)
# Ordinality - {'Norm': 0, 'Feedr/Artery': 1, 'RRA/N': 2, 'PosFeat': 3}
cleaned.loc[cleaned['Condition1'] == 'Norm', 'LotAdjacencyType'] = 0
cleaned.loc[(cleaned['Condition1'] == 'Feedr') | (cleaned['Condition1'] == 'Artery'), 'LotAdjacencyType'] = 1
cleaned.loc[cleaned['Condition1'].str.startswith('RR'), 'LotAdjacencyType'] = 2
cleaned.loc[cleaned['Condition1'].str.startswith('Pos'), 'LotAdjacencyType'] = 3

# 5. OverallQual - (Combine extremes)
# Ordinality - {'below_4': 0, 'Average(4,5,6)': 1, 'above_6': 2}
cleaned.loc[cleaned['OverallQual'] < 4, 'HouseCondition'] = 0
cleaned.loc[cleaned['OverallQual'] <= 6, 'HouseCondition'] = 1
cleaned.loc[cleaned['OverallQual'] >= 7, 'HouseCondition'] = 2

# 6. YearBuilt - Split {MadeBefore1946: 0, MadeAfter1946: 1}
cleaned.loc[cleaned['YearBuilt'] < 1946, 'YrBuilt'] = 0
cleaned.loc[cleaned['YearBuilt'] >= 1946, 'YrBuilt'] = 1

# 7. YearRemodAdd - NEW COLUMN - WasRemodeled
# Process - If the years for YearBuilt and YearRemodAdd are the same, there was no remodel
cleaned.loc[cleaned['YearBuilt'] == cleaned['YearRemodAdd'], 'WasRemodeled'] = 0
cleaned.loc[cleaned['YearBuilt'] != cleaned['YearRemodAdd'], 'WasRemodeled'] = 1

# 8. MasVnrType - (Combine brick-types)
# Ordinality - {'None': 0, 'Brick': 1, 'Stone': 3}
cleaned.loc[cleaned['MasVnrType'] == 'None', 'VeneerType'] = 0
cleaned.loc[cleaned['MasVnrType'].str.startswith('Brk'), 'VeneerType'] = 1
cleaned.loc[cleaned['MasVnrType'] == 'Stone', 'VeneerType'] = 2

# 9. HeatingQC - (Combine Fair and Poor - heating is important!)
# Ordinality - {'Excellent': 0, 'Average': 1, 'Good': 2, 'Poor': 3}
cleaned.loc[cleaned['HeatingQC'] == 'Ex', 'HeatingQuality'] = 0
cleaned.loc[cleaned['HeatingQC'] == 'TA', 'HeatingQuality'] = 1
cleaned.loc[cleaned['HeatingQC'] == 'Gd', 'HeatingQuality'] = 2
cleaned.loc[(cleaned['HeatingQC'] == 'Fa') | (cleaned['HeatingQC'] == 'Po'), 'HeatingQuality'] = 3

# 10. Electrical - (Combine all Fuse types)
# Binary - {'Breaker': 0, 'Fuse': 1}
cleaned.loc[cleaned['Electrical'] == 'SBrkr', 'EleSystem'] = 0
cleaned.loc[(cleaned['Electrical'].str.startswith('Fuse')) | (cleaned['Electrical'] == 'Mix'), 'EleSystem'] = 1

# 11. BsmtFull/HalfBath - NEW COLUMN - BsmtHasBath
cleaned.loc[(cleaned['BsmtFullBath'] == 0) | (cleaned['BsmtHalfBath'] == 0), 'BsmtHasBath'] = 0
cleaned.loc[(cleaned['BsmtFullBath'] > 0) | (cleaned['BsmtHalfBath'] > 0), 'BsmtHasBath'] = 1

# 12. HalfBath - (Combine 1 and 2 to make binary) - HasHalfBath
cleaned.loc[cleaned['HalfBath'] == 0, 'HasHalfBath'] = 0
cleaned.loc[cleaned['HalfBath'] > 0, 'HasHalfBath'] = 1

# 13. BedroomAbvGr - (0-1, 2, 3, 4+)
# Ordinality - {'less_than_2': 0, '2': 1, '3': 2, '4+': 3}
cleaned.loc[cleaned['BedroomAbvGr'] < 2, 'Bedrooms'] = 0
cleaned.loc[cleaned['BedroomAbvGr'] == 2, 'Bedrooms'] = 1
cleaned.loc[cleaned['BedroomAbvGr'] == 3, 'Bedrooms'] = 2
cleaned.loc[cleaned['BedroomAbvGr'] > 3, 'Bedrooms'] = 3

# 14. TotRmsAvbGrd - NEW COLUMN - AdditionalRooms
# Make a new column called RemainingRooms that is the difference between Total Rooms and Bedrooms
# Ordinality - {'less_than_3': 0, '3': 1, '4': 2, '5': 3, 'more_than_5': 4}
cleaned['RemainingRooms'] = cleaned['TotRmsAbvGrd'] - cleaned['BedroomAbvGr']
cleaned.loc[cleaned['RemainingRooms'] < 3, 'AdditionalRooms'] = 0
cleaned.loc[cleaned['RemainingRooms'] == 3, 'AdditionalRooms'] = 1
cleaned.loc[cleaned['RemainingRooms'] == 4, 'AdditionalRooms'] = 2
cleaned.loc[cleaned['RemainingRooms'] == 5, 'AdditionalRooms'] = 3
cleaned.loc[cleaned['RemainingRooms'] > 5, 'AdditionalRooms'] = 4

# 15. Fireplaces - (Combine 2 and 3)
# Ordinality - {'None': 0, '1': 1, '2+': 2}
cleaned.loc[cleaned['Fireplaces'] == 0, 'NumFireplaces'] = 0
cleaned.loc[cleaned['Fireplaces'] == 1, 'NumFireplaces'] = 1
cleaned.loc[cleaned['Fireplaces'] > 1, 'NumFireplaces'] = 2

# 16. GarageCars - (Combine 3 and 4)
# Ordinality - {'0': 0, '1': 1, '2': 2, '3+': 3}
cleaned.loc[cleaned['GarageCars'] == 0, 'GarageAreaByCar'] = 0
cleaned.loc[cleaned['GarageCars'] == 1, 'GarageAreaByCar'] = 1
cleaned.loc[cleaned['GarageCars'] == 2, 'GarageAreaByCar'] = 2
cleaned.loc[cleaned['GarageCars'] > 2, 'GarageAreaByCar'] = 3

# 17. WoodDeckSF - NEW COLUMN - HasDeck
cleaned.loc[cleaned['WoodDeckSF'] == 0, 'HasDeck'] = 0
cleaned.loc[cleaned['WoodDeckSF'] > 0, 'HasDeck'] = 1

# 18. PoolArea - NEW COLUMN - HasPool
cleaned.loc[cleaned['PoolArea'] == 0, 'HasPool'] = 0
cleaned.loc[cleaned['PoolArea'] > 0, 'HasPool'] = 1

# 19. MoSold - Subtract all items by 1
# Ordinality - {'Jan': 0 ... 'Dec': 11}
cleaned['MonthSold'] = cleaned['MoSold'] - 1

# 20. YrSold - Convert years to 0-4 - 2010 might not have concluded at creation of dataset
# Ordinality - {'2006': 0, '2007': 1, '2008': 2, '2009': 3, '2010': 4}
cleaned.loc[cleaned['YrSold'] <= 2006, 'YearSold'] = 0
cleaned.loc[cleaned['YrSold'] == 2007, 'YearSold'] = 1
cleaned.loc[cleaned['YrSold'] == 2008, 'YearSold'] = 2
cleaned.loc[cleaned['YrSold'] == 2009, 'YearSold'] = 3
cleaned.loc[cleaned['YrSold'] == 2010, 'YearSold'] = 4

In [21]:
new_columns = list(cleaned.columns[-21:])

In [22]:
features_encoded = cleaned[new_columns].astype('int64')

In [23]:
df_inter = pd.concat([cleaned[['LotArea', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'SalePrice']], features_encoded], axis=1)
df_inter[condition_LotArea & condition_TotalBsmtSF & condition_1stFlrSF & condition_GrLivArea & condition_SalePrice]

,LotArea,TotalBsmtSF,1stFlrSF,GrLivArea,SalePrice,RegularLotShape,LandIsLvl,LotConfigCL,LotAdjacencyType,HouseCondition,YrBuilt,WasRemodeled,VeneerType,HeatingQuality,EleSystem,BsmtHasBath,HasHalfBath,Bedrooms,RemainingRooms,AdditionalRooms,NumFireplaces,GarageAreaByCar,HasDeck,HasPool,MonthSold,YearSold
0,8450,856,856,1710,208500,1,1,0,0,2,1,0,1,0,0,1,1,2,5,3,0,2,0,0,1,2
1,9600,1262,1262,1262,181500,1,1,3,1,1,1,0,0,0,0,1,0,2,3,1,1,2,1,0,4,1
2,11250,920,920,1786,223500,0,1,0,0,2,1,1,1,0,0,1,1,2,3,1,1,2,0,0,8,2
3,9550,756,961,1717,140000,0,1,1,0,2,0,1,0,2,0,1,0,2,4,2,1,3,0,0,1,0
4,14260,1145,1145,2198,250000,0,1,3,0,2,1,0,1,0,0,1,1,3,5,3,1,3,1,0,11,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,7500,1221,1221,1221,185000,1,1,0,0,2,1,1,0,0,0,1,0,1,4,2,0,2,0,0,9,3
1455,7917,953,953,1647,175000,1,1,0,0,1,1,1,0,0,0,0,1,2,4,2,1,2,0,0,7,1
1457,9042,1152,1188,2340,266500,1,1,0,0,2,0,1,0,0,0,0,0,3,5,3,2,1,0,0,4,4
1458,9717,1078,1078,1078,142125,1,1,0,0,1,1,1,0,2,1,1,0,1,3,1,0,1,1,0,3,4


In [24]:
final_clean_filtered = df_inter[condition_LotArea & condition_TotalBsmtSF & condition_1stFlrSF & condition_GrLivArea & condition_SalePrice]
# final_clean_filtered.to_csv('../cleaned_datasets/train_cleaned.csv', index=False)